In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
# from tensorflow.keras.layers import Conv2D, ReLU, Flatten, Dense, Softmax, BatchNormalization, Input, ZeroPadding2D, Activation
from tensorflow.keras.layers import Input, Conv2D, ReLU, BatchNormalization, Add, MaxPooling2D, Flatten, Dense, Softmax, Activation, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam, Nadam, SGD
import keras
import numpy as np
import h5py

In [2]:
tf.__version__
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [8]:
def dataGenerator(start_index, stop_index):
    def generator():
        indexes = shuff[start_index : stop_index]
        for i in indexes:
            x = np.array(hdf5['Xs'][i,:,:,:])
            y = np.array(hdf5['Ys'][i])
#             x = np.moveaxis(x,0,-1)
            yield x, y
    return generator

In [9]:
hdf5 = h5py.File('./hdf5/kyu_dataset.hdf5','r')
# for j in range (0, 13):
#     print(hdf5['xs'][50][j])
batch_size = 64
dataset_size = hdf5['Xs'].shape[0]
val_start = 0.7
val_stop = 0.8

train_steps = int(dataset_size * (val_start)/batch_size)
val_steps = int(dataset_size * (val_stop - val_start)/batch_size)
print(f"train_steps: {train_steps}, val_steps: {val_steps}")
shuff = np.arange(dataset_size)
np.random.shuffle(shuff)

data_gen = dataGenerator(start_index = 0, stop_index = int(dataset_size * val_start)) # 90% of the complete dataset
dataset = tf.data.Dataset.from_generator(data_gen, 
                                         output_types=(tf.dtypes.float32, tf.dtypes.int32),
                                         output_shapes=(tf.TensorShape((19,19,19)),tf.TensorShape((1))))
dataset = dataset.batch(batch_size, drop_remainder=True)
dataset = dataset.prefetch(tf.data.AUTOTUNE)

data_gen_valid = dataGenerator(start_index=int(dataset_size * val_start), # 10% of the complete dataset
                               stop_index=int(dataset_size * val_stop))
dataset_valid = tf.data.Dataset.from_generator(data_gen_valid, 
                                         output_types=(tf.dtypes.float32, tf.dtypes.int32),
                                         output_shapes=(tf.TensorShape((19,19,19)),tf.TensorShape((1))))
dataset_valid = dataset_valid.batch(batch_size, drop_remainder=True)
dataset_valid = dataset_valid.prefetch(tf.data.AUTOTUNE)


train_steps: 148393, val_steps: 21199


In [10]:
def residual_block(x, filters, kernel_size):
    y = Conv2D(kernel_size=kernel_size,
               filters=filters,
               padding='same')(x)
    y = ReLU()(y)
    y = Conv2D(kernel_size=kernel_size,
               filters=filters,
               padding='same')(y)
    output = Add()([x,y])
    output = ReLU()(output)
    return output

def residual_block_s(x, filters, kernel_size):
    y = Conv2D(kernel_size=kernel_size,
               filters=filters,
               padding='same')(x)
    y = ReLU()(y)
    y = Conv2D(kernel_size=kernel_size,
               filters=filters,
               padding='same')(y)
    x = Conv2D(kernel_size = 1,
              filters=filters)(x)
    output = Add()([x,y])
    output = ReLU()(output)
    return output

def go_res():
    inputs = Input(shape=(19, 19, 19))
    conv5x5 = Conv2D(kernel_size=5,
                     filters=64,
                     padding="same",
                     name='conv5x5')(inputs)
    conv1x1 = Conv2D(kernel_size=1,
                     filters=64,
                     padding="same",
                     name='conv1x1')(inputs)
    outputs = Add()([conv5x5, conv1x1])
    outputs = ReLU()(outputs)
    outputs = residual_block(x=outputs,
                             filters=64,
                             kernel_size=3)
#     outputs = residual_block(x=outputs,
#                              filters=64,
#                              kernel_size=3)
#     outputs = residual_block(x=outputs,
#                              filters=64,
#                              kernel_size=3)
#     outputs = residual_block(x=outputs,#
#                              filters=64,
#                              kernel_size=3)
#     outputs = residual_block(x=outputs,
#                              filters=64,
#                              kernel_size=3)
#     outputs = residual_block(x=outputs,
#                              filters=64,
#                              kernel_size=3)
#     outputs = residual_block(x=outputs,
#                              filters=64,
#                              kernel_size=3)
#     outputs = residual_block(x=outputs,
#                              filters=64,
#                              kernel_size=3)
#     outputs = residual_block(x=outputs,
#                              filters=64,
#                              kernel_size=3)
#     outputs = residual_block(x=outputs,
#                              filters=64,
#                              kernel_size=3)
#     outputs = residual_block(x=outputs,
#                              filters=64,
#                              kernel_size=3)
#     outputs = residual_block(x=outputs,
#                              filters=64,
#                              kernel_size=3)
#     outputs = residual_block(x=outputs,
#                              filters=64,
#                              kernel_size=3)
#     outputs = residual_block(x=outputs,#
#                              filters=64,
#                              kernel_size=3)
#     outputs = residual_block(x=outputs,
#                              filters=64,
#                              kernel_size=3)
#     outputs = residual_block(x=outputs,
#                              filters=64,
#                              kernel_size=3)
#     outputs = residual_block(x=outputs,
#                              filters=64,
#                              kernel_size=3)
#     outputs = Conv2D(kernel_size=3,
#                      filters=1,
#                      padding="same")(outputs)
#     outputs = ReLU()(outputs)
#     outputs = Flatten()(outputs)
#     outputs = Softmax()(outputs)
    outputs = GlobalAveragePooling2D()(outputs)
    outputs = Dense(361, activation='softmax')(outputs)
    model = Model(inputs, outputs)
    
    opt = Adam(learning_rate=0.0001)
    model.compile(optimizer=opt,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model


model = go_res()
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 19, 19, 19)  0           []                               
                                ]                                                                 
                                                                                                  
 conv5x5 (Conv2D)               (None, 19, 19, 64)   30464       ['input_2[0][0]']                
                                                                                                  
 conv1x1 (Conv2D)               (None, 19, 19, 64)   1280        ['input_2[0][0]']                
                                                                                                  
 add_2 (Add)                    (None, 19, 19, 64)   0           ['conv5x5[0][0]',          

In [11]:
# callback1 = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, 
#                                 patience=1, verbose=0, mode='min'
#                                  , restore_best_weights=True)
callback1 = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                patience=2, verbose=0, mode='max'
                                 , restore_best_weights=True)
callback2 = keras.callbacks.ModelCheckpoint('./models/model_kyu_pp_{epoch:02d}_{val_accuracy:.3f}.h5', 
                                            monitor='val_accuracy', 
                                            verbose=0, save_best_only=False, save_weights_only=False, 
                                            mode='max', save_freq="epoch")

In [12]:
history = model.fit(
    dataset,
    epochs = 500,
#     steps_per_epoch = train_steps,
    validation_data = dataset_valid,
#     validation_steps = val_steps,
    callbacks = [callback1, callback2]
)

Epoch 1/500
     18/Unknown - 11s 431ms/step - loss: 5.8925 - accuracy: 0.0017    

KeyboardInterrupt: 

In [7]:
model.save('./models/model_kyu_resnet50_withpp.h5')

In [8]:
result = model.evaluate(
    dataset_valid,
    steps = val_steps
)

11374/21176 [===============>..............] - ETA: 43:34 - loss: 1.6519 - accuracy: 0.5500

KeyboardInterrupt: 

In [ ]:
model.save('./models/model_kyu_resnet50_withpp_1.h5')

In [27]:
history = model.fit(
    dataset,
    epochs = 1,
    validation_data = val_dataset
)

95270/95270 [==============================] - 14285s 150ms/step - loss: 2.0121 - accuracy: 0.5265


In [28]:
model.save('./models/model_kyu_resnet_2.h5')

In [33]:
history = model.fit(
    dataset,
    epochs = 1,
    validation_data = val_dataset
)

42/42 [==============================] - 8s 188ms/step - loss: 1.8230 - accuracy: 0.5787 - val_loss: 1.9512 - val_accuracy: 0.5478


In [ ]:
model.save('./models/model_kyu_resnet_3.h5')

In [ ]:
history = model.fit(
    dataset,
    epochs = 1
)

In [ ]:
model.save('./models/model_kyu_resnet_4.h5')

In [ ]:
history = model.fit(
    dataset,
    epochs = 1
)

In [ ]:
model.save('./models/model_kyu_10_15_f128_5.h5')

In [ ]:
history = model.fit(
    dataset,
    epochs = 1
)

In [ ]:
model.save('./models/model_kyu_10_15_f128_6.h5')

In [ ]:
history = model.fit(
    dataset,
    epochs = 1
)

In [ ]:
model.save('./models/model_kyu_10_15_f128_7.h5')

In [ ]:
history = model.fit(
    dataset,
    epochs = 1
)

In [ ]:
model.save('./models/model_kyu_10_15_f128_8.h5')

In [ ]:
history = model.fit(
    dataset,
    epochs = 1
)

In [ ]:
model.save('./models/model_kyu_10_15_f128_9.h5')

In [ ]:
history = model.fit(
    dataset,
    epochs = 1
)

In [ ]:
model.save('./models/model_kyu_10_14_11.h5')

## ALL DONE!

For using the model and creating a submission file, follow the notebook **Create Public Upload CSV.ipynb**

# End of Tutorial

You are free to use more modern NN architectures, a better pre-processing, feature extraction methods to achieve much better accuracy!